# Język Python - Wykład 5.

## Wyrażenia regularne

In [ ]:
import re

<img width=800 src="files/L5_img/automata.png">

<img width=300 src="files/L5_img/plus.png">

## Metaznaki

## Użycie

In [ ]:
p = re.compile('ab|bc')
print p

In [ ]:
print p.match('bc')

In [ ]:
print p.match('ba')

In [ ]:
re.search('b','abc')

In [ ]:
re.match('b','abc')

In [ ]:
re.match('a','abc')

In [ ]:
re.split(r'\W+', 'Words, words, words.')

In [ ]:
re.split(r'(\W+)', 'Words, words, words.')

In [ ]:
p = re.compile('(ab)*')
p.match('ababababab').span()

In [ ]:
p = re.compile('a/{,3}b')
print p.match('ab')
print p.match('a/b')
print p.match('a//b')
print p.match('a///b')
print p.match('a////b')

In [ ]:
print re.search(r'\bfoo\b','bar foo baz').group()

In [ ]:
print re.findall(r'[0-9]','a1b2c3d4e5')

In [ ]:
for m in re.finditer(r'[0-9]','a1b2c3d4e5'):
    print m.group(0),m.span()

## Notacja „raw string”

In [ ]:
re.match(r"\W(.)\1\W", " ff ").group()

In [ ]:
re.match("\\W(.)\\1\\W", " ff ").group()

## Niezachłanne dopasowania

In [ ]:
s = '<html><head><title>Title</title>'

In [ ]:
print re.match('<.*>', s).group()

In [ ]:
print re.match('<.*?>', s).group()

## Grupowanie

In [ ]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')

In [ ]:
m.group(0)

In [ ]:
m.group(1)

In [ ]:
m.group(2)

In [ ]:
#Match dates formatted like MM/DD/YYYY, MM-DD-YY,...
import re

date = '01/12/2013'

regex = re.compile(r'^(?P<day>\d\d)[-/](?P<month>\d\d)[-/](?P<year>\d\d(?:\d\d)?)$')

match = regex.match(date)

In [ ]:
print match.group(0),match.group(1),match.group(2),match.group(3)

In [ ]:
print match.group('day'),match.group('month'),match.group('year')

In [ ]:
print match.groups()

In [ ]:
print regex.groupindex

In [ ]:
re.match(r'(ala) \1','ala ala').group(0)

## Extension Notation (?...)

### Powielenie (?P=<name>)

In [ ]:
p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)')

In [ ]:
p.match('ala ala').group()

In [ ]:
p = re.compile(r'(?P<word>\b\w+)\s+\1')

In [ ]:
p.match('ala ala').group()

### Nieprzechwytująca grupa (?:foo)

In [ ]:
re.match(r'(\w+@\w+(?:\.\w+)+)','korzycki@agh.edu.pl').groups()

### Positive Lookahead (?=...)

In [ ]:
re.match(r'(\w+(?=@\w+(?:.\w+)+))','korzycki@agh.edu.pl').group()

### Negative Lookahead (?!...)

In [1]:
s='plik.py'
re.match(r'(.*)[.](?!bat$|exe$).*$',s).group(1)

NameError: name 're' is not defined

### Positive Lookbehind

In [ ]:
s='From: korzycki@agh.edu.pl'
re.search(r'(?<=From: )(.*)',s).group()

### Negative Lookbehind 

In [ ]:
s='From: korzycki@agh.edu.pl'
re.search(r'(?<!^Subject)(?<=: )(.*)',s).group()

### Warunkowe wyrażenie (?(id/name)yes-pattern|no-pattern)

In [ ]:
import re
p = re.compile('(<)?(\w+@\w+(?:\.\w+)+)(?(1)>)')

In [ ]:
p.match('<user@host.com>').group()

In [ ]:
p.match('user@host.com').group()

In [ ]:
print p.match('<user@host.com')

## Flagi

### re.VERBOSE

In [ ]:
pat = re.compile(r"\s*(?P<header>[^:]+)\s*:(?P<value>.*?)\s*$")

In [ ]:
pat.match('From: korzycki@agh.edu.pl').groups()

In [ ]:
pat = re.compile(r"""
    \s*               # Skip leading whitespace
    (?P<header>[^:]+) # Header name
    \s* :             # Whitespace, and a colon
    (?P<value>.*?)    # The header's value -- *? used to
                      # lose the following trailing whitespace
    \s*$
""", re.VERBOSE)


In [ ]:
pat.match('From: korzycki@agh.edu.pl').groups()

### re.IGNORECASE

In [ ]:
re.match(r'(?i)ala','Ala').group()

### re.MULTILINE re.DOTALL

In [ ]:
s="""Ala ma
kota"""

print re.match(r'^A.*ta$',s)
print re.match(r'(?ms)^A.*ta$',s)

## Podstawienie

In [ ]:
#urlify - turn URLs into HTML links
import re

text = 'Check the web site, http://www.oreilly.com/catalog/regexppr.'

pattern = r'''
    \b                                    # start at word boundary
    (                                     # capture to \1
    (https?|telnet|gopher|file|wais|ftp) :# resource and colon
    [\w/#~:.?+=&%@!\-] +?                 # one or more valid chars
                                          # take little as possible
    )
    (?=                                   # lookahead
    [.:?\-] *                             # for possible punc
    (?: [^\w/#~:.?+=&%@!\-]               # invalid character
    | $ )                                 # or end of string
    )'''

regex = re.compile(pattern,re.IGNORECASE + re.VERBOSE)
result = regex.sub(r'<a href="\1">\1</a>', text)

print result

### Podstawienie funkcją

In [ ]:
import random
def repl(m):
    inner_word = list(m.group(2))
    random.shuffle(inner_word)
    return m.group(1) + "".join(inner_word) + m.group(3)


In [ ]:
text = "Professor Abdolmalek, please report your absences promptly."
re.sub("(\w)(\w+)(\w)", repl, text)

In [ ]:
indent = lambda s: reduce(min,map(len,re.findall('(?m)^ *(?=\S)',s)))
flush_left = lambda s: re.sub('(?m)^ {%d}' % indent(s),'',s)

In [ ]:
s="""    
    Bardzo
       roznie
     wciety
    tekst 
"""
print s
print
print flush_left(s)